In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

# Load the dataset
file_path = '/content/ex.csv'
data = pd.read_csv(file_path)


# Step 1: Data Cleaning
data['Singer/Artists'].fillna('Unknown', inplace=True)
data['Album/Movie'].fillna('Unknown', inplace=True)
data['User-Rating'] = data['User-Rating'].str.split('/').str[0].astype(float)

# Combine relevant text features into a single column
data['combined_features'] = (
    data['Singer/Artists'] + ' ' + data['Genre'] + ' ' + data['Album/Movie']
)

# Vectorize the combined features
vectorizer = TfidfVectorizer(stop_words='english')
feature_matrix = vectorizer.fit_transform(data['combined_features'])

# Compute the similarity matrix
similarity_matrix = cosine_similarity(feature_matrix)

# Step 2: Function to Recommend Songs Based on Input Details
def recommend_songs_from_input(song_details, num_recommendations=5):
    """
    Recommend songs based on user-inputted details.

    :param song_details: Dictionary containing song details (attributes)
    :param num_recommendations: Number of recommendations to return
    :return: List of recommended songs
    """
    # Create a feature string based on the input details
    input_features = (
        song_details['Singer/Artists'] + ' ' +
        song_details['Genre'] + ' ' +
        song_details['Album/Movie']
    )

    # Vectorize the input features
    input_vector = vectorizer.transform([input_features])

    # Compute similarity scores with all songs in the dataset
    similarity_scores = cosine_similarity(input_vector, feature_matrix).flatten()

    # Get the indices of the most similar songs
    similar_indices = similarity_scores.argsort()[-num_recommendations - 1 :][::-1]

    # Exclude the input song itself if it exists in the dataset
    recommended_songs = [
        data.iloc[i]['Song-Name']
        for i in similar_indices
        if similarity_scores[i] > 0
    ]

    return recommended_songs[:num_recommendations]

# Step 3: Get Custom User Input
print("Enter details of a song:")
singer_artists = input("Singer/Artists: ")
genre = input("Genre: ")
album_movie = input("Album/Movie: ")

user_input = {
    "Singer/Artists": singer_artists,
    "Genre": genre,
    "Album/Movie": album_movie
}

# Get recommendations
recommendations = recommend_songs_from_input(user_input, num_recommendations=5)

# Display recommendations
print("\nRecommended Songs:")
for i, song in enumerate(recommendations, 1):
    print(f"{i}. {song}")


Enter details of a song:
Singer/Artists: neha kakkar
Genre: romantic
Album/Movie: luka chupi

Recommended Songs:
1. Coca Cola
2. O Janiya
3. Mile Ho Tum (Reprise)
4. Akkad Bakkad
5. Mohabbat Nasha Hai
